In [ ]:
import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist
from sklearn.preprocessing import LabelEncoder

def causal_graphical_model(ad_campaign, user_segment, region):
    """
    Causal Graphical Model: Define generative process for advertising scenario
    
    Input:
    - ad_campaign: Ad campaign type (encoded)
    - user_segment: User segment (encoded)
    - region: Region (encoded)
    
    Output:
    - Dictionary of generated data points
    
    Function:
    - Implement Structural Causal Model (SCM)
    - Define causal mechanisms between variables
    - Generate data consistent with causal graph
    """
    # Ensure inputs are scalar tensors
    ad_idx = ad_campaign if isinstance(ad_campaign, int) else ad_campaign.item()
    user_idx = user_segment if isinstance(user_segment, int) else user_segment.item()
    region_idx = region if isinstance(region, int) else region.item()
    
    # Ad intensity determined by ad type (fixed indexing issue)
    intensity_probs = torch.tensor([
        [0.3, 0.4, 0.3],  # Social_Media_Promo
        [0.4, 0.3, 0.3],  # Search_Engine_Ad  
        [0.2, 0.5, 0.3],  # Email_Campaign
        [0.1, 0.4, 0.5]   # Video_Commercial
    ])
    
    ad_intensity = pyro.sample(
        "ad_intensity",
        dist.Categorical(probs=intensity_probs[ad_idx])
    )
    
    # Click behavior determined by ad type, intensity, and user characteristics
    ad_effects = torch.tensor([0.5, 0.8, 1.0, 1.2])  # Base effects of different ad types
    intensity_effects = torch.tensor([0.1, 0.3, 0.5])  # Effects of different ad intensities
    user_effects = torch.tensor([0.2, 0.4, 0.3, 0.5, 0.6])  # Effects of different user segments
    
    click_logits = (
        ad_effects[ad_idx] + 
        intensity_effects[ad_intensity] + 
        user_effects[user_idx] * 0.5
    )
    
    # Expand to 4 click behavior categories
    click_logits_expanded = torch.tensor([
        click_logits * 0.5,    # No_Click
        click_logits * 0.8,    # Quick_View  
        click_logits * 1.2,    # Detailed_Engagement
        click_logits * 1.5     # Shared_Content
    ])
    
    click_behavior = pyro.sample(
        "click_behavior",
        dist.Categorical(logits=click_logits_expanded)
    )
    
    # Purchase decision determined by click behavior and all preceding variables
    click_purchase_effects = torch.tensor([1.0, 1.5, 2.0, 2.5])  # Click behavior impact on purchase
    intensity_purchase_effects = torch.tensor([0.3, 0.5, 0.7])  # Ad intensity impact on purchase
    ad_purchase_effects = torch.tensor([0.2, 0.4, 0.3, 0.5])   # Ad type impact on purchase
    
    purchase_logits = (
        click_purchase_effects[click_behavior] +
        intensity_purchase_effects[ad_intensity] +
        ad_purchase_effects[ad_idx] * 0.3
    )
    
    # Expand to 4 purchase decision categories
    purchase_logits_expanded = torch.tensor([
        purchase_logits * 0.5,  # No_Purchase
        purchase_logits * 0.8,  # Small_Order
        purchase_logits * 1.2,  # Medium_Order  
        purchase_logits * 1.5   # Large_Order
    ])
    
    purchase_decision = pyro.sample(
        "purchase_decision",
        dist.Categorical(logits=purchase_logits_expanded)
    )
    
    # Satisfaction determined by purchase decision and overall experience
    purchase_satisfaction_effects = torch.tensor([2.0, 1.5, 1.0, 0.5])  # Purchase decision impact on satisfaction
    click_satisfaction_effects = torch.tensor([1.0, 1.8, 2.5, 3.0])     # Click behavior impact on satisfaction
    
    satisfaction_logits = (
        purchase_satisfaction_effects[purchase_decision] +
        click_satisfaction_effects[click_behavior] * 0.5
    )
    
    # Expand to 5 satisfaction categories
    satisfaction_logits_expanded = torch.tensor([
        satisfaction_logits * 0.3,  # Very_Unsatisfied
        satisfaction_logits * 0.6,  # Somewhat_Unsatisfied
        satisfaction_logits * 1.0,  # Neutral
        satisfaction_logits * 1.3,  # Somewhat_Satisfied  
        satisfaction_logits * 1.6   # Very_Satisfied
    ])
    
    customer_satisfaction = pyro.sample(
        "customer_satisfaction",
        dist.Categorical(logits=satisfaction_logits_expanded)
    )
    
    return {
        'ad_intensity': ad_intensity,
        'click_behavior': click_behavior,
        'purchase_decision': purchase_decision,
        'customer_satisfaction': customer_satisfaction
    }

def do_intervention_analysis(df, intervention_ad_campaign):
    """
    Perform intervention analysis: What happens if we force-change ad campaigns
    
    Input:
    - df: Raw data
    - intervention_ad_campaign: Ad campaign type for intervention
    
    Output:
    - intervention_results: Comparison results before and after intervention
    
    Function:
    - Implement do-calculus intervention
    - Calculate Average Treatment Effect (ATE)
    - Compare outcome distributions before and after intervention
    """
    print(f"\n=== Intervention Analysis: Force all users to receive {intervention_ad_campaign} ads ===")
    
    # Encoders
    encoders = {}
    for col in ['ad_campaign', 'click_behavior', 'purchase_decision']:
        le = LabelEncoder()
        df[col + '_encoded'] = le.fit_transform(df[col])
        encoders[col] = le
    
    # Original data statistics - fixed purchase rate calculation
    original_purchase_rates = {}
    for campaign in df['ad_campaign'].unique():
        campaign_data = df[df['ad_campaign'] == campaign]
        # Calculate actual purchase rate (proportion where purchase decision is not "No_Purchase")
        purchase_rate = (campaign_data['purchase_decision'] != 'No_Purchase').mean()
        original_purchase_rates[campaign] = purchase_rate
    
    print("Original data purchase rates:")
    for campaign, rate in original_purchase_rates.items():
        count = len(df[df['ad_campaign'] == campaign])
        print(f"  {campaign}: {rate:.3f} (n={count})")
    
    # Simulate intervention
    intervention_ad_encoded = encoders['ad_campaign'].transform([intervention_ad_campaign])[0]
    
    intervention_effects = []
    user_segment_encoder = LabelEncoder()
    user_segments_encoded = user_segment_encoder.fit_transform(df['user_segment'].unique())
    
    for user_segment in df['user_segment'].unique():
        segment_data = df[df['user_segment'] == user_segment]
        user_encoded = user_segment_encoder.transform([user_segment])[0]
        
        # Use typical region for this user segment
        typical_region = segment_data['region'].mode()[0]
        region_encoder = LabelEncoder()
        region_encoder.fit(df['region'])
        region_encoded = region_encoder.transform([typical_region])[0]
        
        # Simulate intervention effects
        n_simulations = 50
        intervention_purchases = []
        
        for _ in range(n_simulations):
            with pyro.poutine.trace() as trace:
                intervention_result = causal_graphical_model(
                    intervention_ad_encoded, user_encoded, region_encoded
                )
            # Check if purchase occurred (purchase decision not 0, i.e., not "No_Purchase")
            did_purchase = intervention_result['purchase_decision'] > 0
            intervention_purchases.append(did_purchase.item() if isinstance(did_purchase, torch.Tensor) else did_purchase)
        
        predicted_purchase_rate = np.mean(intervention_purchases)
        original_purchase_rate = (segment_data['purchase_decision'] != 'No_Purchase').mean()
        
        intervention_effects.append({
            'user_segment': user_segment,
            'original_rate': original_purchase_rate,
            'intervention_rate': predicted_purchase_rate,
            'treatment_effect': predicted_purchase_rate - original_purchase_rate
        })
    
    print(f"\nIntervention effect analysis ({intervention_ad_campaign}):")
    for effect in intervention_effects:
        print(f"  {effect['user_segment']}: {effect['treatment_effect']:+.3f} "
              f"({effect['original_rate']:.3f} → {effect['intervention_rate']:.3f})")
    
    # Calculate Average Treatment Effect
    ate = np.mean([effect['treatment_effect'] for effect in intervention_effects])
    print(f"\nAverage Treatment Effect (ATE): {ate:+.3f}")
    
    return intervention_effects

def counterfactual_analysis(df, user_id, alternative_ad_campaign):
    """
    Perform counterfactual analysis: What if specific user received different ad strategy
    
    Input:
    - df: Raw data
    - user_id: User ID to analyze
    - alternative_ad_campaign: Alternative ad strategy
    
    Output:
    - counterfactual_result: Counterfactual analysis results
    
    Function:
    - Build counterfactuals based on actual observed data
    - Estimate counterfactual outcomes using causal model
    - Compare actual results with counterfactual results
    """
    print(f"\n=== Counterfactual Analysis: What if user {user_id} received {alternative_ad_campaign} ads ===")
    
    # Get user's actual data
    user_data = df[df['user_id'] == user_id].iloc[0]
    print(f"User actual data:")
    print(f"  User segment: {user_data['user_segment']}")
    print(f"  Actual ad: {user_data['ad_campaign']}")
    print(f"  Click behavior: {user_data['click_behavior']}")
    print(f"  Purchase decision: {user_data['purchase_decision']}")
    print(f"  Satisfaction: {user_data['customer_satisfaction']}")
    
    # Encoders
    encoders = {}
    for col in ['user_segment', 'ad_campaign', 'region', 'click_behavior', 'purchase_decision', 'customer_satisfaction']:
        le = LabelEncoder()
        le.fit(df[col])
        encoders[col] = le
    
    # Encode user characteristics
    user_segment_encoded = encoders['user_segment'].transform([user_data['user_segment']])[0]
    region_encoded = encoders['region'].transform([user_data['region']])[0]
    actual_ad_encoded = encoders['ad_campaign'].transform([user_data['ad_campaign']])[0]
    alternative_ad_encoded = encoders['ad_campaign'].transform([alternative_ad_campaign])[0]
    
    # Simulate counterfactual scenarios
    n_simulations = 200
    counterfactual_results = []
    
    for _ in range(n_simulations):
        cf_result = causal_graphical_model(
            alternative_ad_encoded, user_segment_encoded, region_encoded
        )
        counterfactual_results.append({
            'purchase_decision': cf_result['purchase_decision'].item(),
            'customer_satisfaction': cf_result['customer_satisfaction'].item(),
            'click_behavior': cf_result['click_behavior'].item()
        })
    
    # Analyze results
    actual_purchase_encoded = encoders['purchase_decision'].transform([user_data['purchase_decision']])[0]
    actual_satisfaction_encoded = encoders['customer_satisfaction'].transform([user_data['customer_satisfaction']])[0]
    actual_click_encoded = encoders['click_behavior'].transform([user_data['click_behavior']])[0]
    
    cf_purchases = [r['purchase_decision'] for r in counterfactual_results]
    cf_satisfactions = [r['customer_satisfaction'] for r in counterfactual_results]
    cf_clicks = [r['click_behavior'] for r in counterfactual_results]
    
    print(f"\nCounterfactual results ({alternative_ad_campaign}):")
    
    print(f"  Click behavior distribution:")
    for i, behavior in enumerate(encoders['click_behavior'].classes_):
        prob = np.mean([1 if click == i else 0 for click in cf_clicks])
        actual_indicator = " ← Actual" if i == actual_click_encoded else ""
        print(f"    {behavior}: {prob:.3f}{actual_indicator}")
    
    print(f"  Purchase decision distribution:")
    for i, decision in enumerate(encoders['purchase_decision'].classes_):
        prob = np.mean([1 if purchase == i else 0 for purchase in cf_purchases])
        actual_indicator = " ← Actual" if i == actual_purchase_encoded else ""
        print(f"    {decision}: {prob:.3f}{actual_indicator}")
    
    print(f"  Satisfaction distribution:")
    for i, satisfaction in enumerate(encoders['customer_satisfaction'].classes_):
        prob = np.mean([1 if sat == i else 0 for sat in cf_satisfactions])
        actual_indicator = " ← Actual" if i == actual_satisfaction_encoded else ""
        print(f"    {satisfaction}: {prob:.3f}{actual_indicator}")
    
    # Calculate counterfactual gains
    purchase_gain = (np.array(cf_purchases) > actual_purchase_encoded).mean() - (np.array(cf_purchases) < actual_purchase_encoded).mean()
    satisfaction_gain = (np.array(cf_satisfactions) > actual_satisfaction_encoded).mean() - (np.array(cf_satisfactions) < actual_satisfaction_encoded).mean()
    
    print(f"\nCounterfactual gain assessment:")
    print(f"  Purchase decision improvement probability: {purchase_gain:+.3f}")
    print(f"  Satisfaction improvement probability: {satisfaction_gain:+.3f}")
    
    return {
        'actual_data': user_data.to_dict(),
        'counterfactual_results': counterfactual_results,
        'purchase_gain': purchase_gain,
        'satisfaction_gain': satisfaction_gain
    }

def run_causal_analysis(df):
    """
    Run complete causal inference analysis pipeline
    
    Input:
    - df: DataFrame containing advertising scenario data
    
    Output:
    - Dictionary containing all analysis results
    
    Function:
    - Integrate generative model, intervention analysis, and counterfactual analysis
    - Provide complete causal inference insights
    """
    print("Starting causal inference analysis...")
    
    # Set random seeds for reproducible results
    torch.manual_seed(42)
    np.random.seed(42)
    
    # 1. Intervention analysis
    intervention_results = do_intervention_analysis(df, "Video_Commercial")
    
    # 2. Counterfactual analysis (select few typical users)
    sample_users = df['user_id'].sample(2).tolist()  # Reduce sample size for speed
    counterfactual_results = {}
    
    for user_id in sample_users:
        cf_result = counterfactual_analysis(df, user_id, "Social_Media_Promo")
        counterfactual_results[user_id] = cf_result
    
    # 3. Causal effect summary
    print("\n=== Causal Inference Summary ===")
    avg_treatment_effect = np.mean([effect['treatment_effect'] for effect in intervention_results])
    
    if avg_treatment_effect > 0:
        print(f"1. Intervention analysis shows Video_Commercial increases purchase rate by {avg_treatment_effect:.3f} on average")
    else:
        print(f"1. Intervention analysis shows Video_Commercial decreases purchase rate by {abs(avg_treatment_effect):.3f} on average")
    
    print("2. Counterfactual analysis reveals individual user sensitivity to different ad strategies")
    print("3. Model successfully identifies causal path: Ad Type → Click Behavior → Purchase Decision")
    
    return {
        'intervention_results': intervention_results,
        'counterfactual_results': counterfactual_results,
        'average_treatment_effect': avg_treatment_effect
    }

# Run causal inference analysis
if __name__ == "__main__":
    # Load data
    df = pd.read_csv('causal_advertising_data.csv')
    
    # Run causal analysis
    causal_results = run_causal_analysis(df)